In [1]:
import math

import pandas as pd
from neupy import algorithms
import numpy as np
import json
from tqdm import tqdm
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from numpy import sqrt
import math

/Users/tomas.dallas/itba/binance-bot/venv/lib/python3.8/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
def ret(n, n_1):
    return (n_1 - n) / n


PREDICTIONS = 1000
# prices every 15 seconds
# df = pd.read_csv("prices.csv")
f = open("BTC_USDT-3m.json")
df = pd.DataFrame.from_records(json.load(f), columns=['timestamp', 'open', 'high', 'low', 'close', 'vol'])
# df.columns = ['price']
df['ret_c'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
df['ret_o'] = (df['open'] - df['open'].shift(1)) / df['open'].shift(1)

df['ret_h'] = (df['high'] - df['high'].shift(1)) / df['high'].shift(1)
df['ret_l'] = (df['low'] - df['low'].shift(1)) / df['low'].shift(1)
df['ret'] = (df['high'].shift(-1) - df['close']) / df['close']
df['ret_s'] = df['ret'].apply(lambda x:  np.sign(x))
df = df.iloc[0:PREDICTIONS + 3]


def perc_dif(actual, prediction):
    return np.abs(prediction - actual) / prediction

In [3]:
df

,timestamp,open,high,low,close,vol,ret_c,ret_o,ret_h,ret_l,ret,ret_s
0,1661040000000,21140.07,21163.88,21136.70,21162.06,298.59140,NaN,NaN,NaN,NaN,0.000376,1.0
1,1661040180000,21162.06,21170.01,21119.90,21128.62,383.42144,-0.001580,0.001040,0.000290,-0.000795,0.001721,1.0
2,1661040360000,21128.62,21164.99,21118.71,21163.56,319.63957,0.001654,-0.001580,-0.000237,-0.000056,0.001958,1.0
3,1661040540000,21163.56,21205.00,21161.60,21189.24,385.29906,0.001213,0.001654,0.001890,0.002031,0.000071,1.0
4,1661040720000,21189.24,21190.74,21158.10,21168.10,263.47068,-0.000998,0.001213,-0.000672,-0.000165,0.002209,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
998,1661219640000,21317.88,21320.96,21303.54,21304.06,127.66907,-0.000685,-0.000067,-0.000289,0.000618,0.000701,1.0
999,1661219820000,21304.06,21319.00,21301.96,21306.86,171.74079,0.000131,-0.000648,-0.000092,-0.000074,0.000338,1.0
1000,1661220000000,21306.86,21314.07,21300.00,21302.51,185.11035,-0.000204,0.000131,-0.000231,-0.000092,0.000294,1.0
1001,1661220180000,21303.71,21308.78,21295.86,21299.96,233.84119,-0.000120,-0.000148,-0.000248,-0.000194,0.001063,1.0


In [4]:
# OJO CON NDTR=2 y std=0.0029
# OJO CON NDTR=4 y std=0.000075
sq_errors = []
std = 0.0003
hit_rates = []
for NDTR in np.arange(2, 10):
    for std in tqdm(np.concatenate((np.arange(0.001, 1, 0.01), np.arange(0.00001, 0.001, 0.00005), np.arange(1, 10, 0.5)))):
        nans = 0
        actuals = []
        predictions = []
        hits = 0
        for i in np.arange(NDTR, NDTR + PREDICTIONS + 1):
            small_df = df.iloc[i: i + NDTR + 1].copy(deep=True)[['ret_o', 'ret']].reset_index(
                drop=True)
            small_df['ret_o'] = small_df['ret_o'] / small_df['ret_o'].mean()
            small_df['ret'] = small_df['ret'] / small_df['ret'].mean()
            x_train = small_df.iloc[0:NDTR][['ret_o']].values
            y_train = small_df['ret'].iloc[0:NDTR].values
            grnn = algorithms.GRNN(std=std, verbose=False)
            grnn.train(x_train, y_train)
            if len(small_df) > NDTR:
                x_predict = small_df.iloc[NDTR][['ret_o']].values
                prediction = grnn.predict([x_predict])
                if math.isnan(prediction):
                    nans += 1
                    prediction = 0
                else:
                    actual = small_df['ret'].iloc[NDTR]
                    prediction = prediction[0]
                    if np.sign(actual) == np.sign(prediction):
                        hits += 1
                    actuals.append(actual)
                    predictions.append(prediction)
        if len(actuals) > 900:
            hit_rates.append([hits / len(actuals), std, NDTR, len(actuals)])
            # fig = make_subplots()
            # fig.add_trace(
            #     go.Scatter(x=actuals, y=predictions,
            #                name="Retornos", mode='lines+markers'))
            # title= "GRNN NDTR=" + str(NDTR) + "| std=" + str(std) + "|HitRate=" + "{:.2%}".format(hits/len(actuals))
            # fig.update_layout(title=title, xaxis_title="Real", yaxis_title="Prediction")
            # # fig.show()
            # fig.write_image("grnn/prediction_vs_real_NDTR" + str(NDTR) + "_std" + str(std) + "_nosign_nonorm.png")

hit_df = pd.DataFrame(hit_rates, columns=['hit_rate', 'std', 'NDTR', 'predictions'])
hit_df.to_csv("grnn/hits_grnn_nosign_high.csv", header=None, index=None)

hit_df = hit_df[hit_df['predictions'] > 900]
fig = make_subplots()

for NDTR in hit_df['NDTR'].unique():
    actual_hdf = hit_df[hit_df['NDTR'] == NDTR]
    fig.add_trace(
        go.Scatter(x=actual_hdf['std'], y=actual_hdf['hit_rate'].apply(lambda x: 1 - x), name="NDTR=" + str(NDTR),
                   mode="markers+lines"))

fig.update_layout(yaxis_tickformat=".2%", title="Hit rate of prediction for a given std", xaxis_title="STD",
                  yaxis_title="Hit rate", xaxis_tickformat="none")
fig.write_image("grnn/hitrate_nosign_rethigh.png")

fig.show()

  1%|▏         | 2/138 [00:08<10:05,  4.45s/it]


KeyboardInterrupt: 